In [47]:
from merlin.io import Dataset

train_df = Dataset("./data/ebnerd_demo/train/history.parquet", engine="parquet")
schema = train_df.schema

/home/scur1565/.local/lib/python3.11/site-packages/merlin/io/dataset.py:267: UserWarning: Initializing an NVTabular Dataset in CPU mode.This is an experimental feature with extremely limited support!
  warnings.warn(


In [48]:
schema

,name,tags,dtype,is_list,is_ragged,properties.value_count.min,properties.value_count.max
0,user_id,(),"DType(name='uint32', element_type=<ElementType...",False,False,NaN,NaN
1,impression_time_fixed,(),"DType(name='datetime64[ns]', element_type=<Ele...",True,True,0.0,NaN
2,scroll_percentage_fixed,(),"DType(name='float32', element_type=<ElementTyp...",True,True,0.0,NaN
3,article_id_fixed,(),"DType(name='int32', element_type=<ElementType....",True,True,0.0,NaN
4,read_time_fixed,(),"DType(name='float32', element_type=<ElementTyp...",True,True,0.0,NaN


In [49]:
# You can select a subset of features for training
schema = schema.select_by_name(
    [
        'user_id',
        'article_id_fixed', 
        'read_time_fixed'
    ]
)

In [50]:
schema

,name,tags,dtype,is_list,is_ragged,properties.value_count.min,properties.value_count.max
0,user_id,(),"DType(name='uint32', element_type=<ElementType...",False,False,NaN,NaN
1,article_id_fixed,(),"DType(name='int32', element_type=<ElementType....",True,True,0.0,NaN
2,read_time_fixed,(),"DType(name='float32', element_type=<ElementTyp...",True,True,0.0,NaN


In [44]:
from transformers4rec import torch as tr
from transformers4rec.torch.ranking_metric import NDCGAt, RecallAt

max_sequence_length, d_model = 20, 64

inputs = tr.TabularSequenceFeatures.from_schema(
        schema,
        max_sequence_length=max_sequence_length,
        continuous_projection=d_model,
        masking="mlm",
        d_output=100,
)

KeyError: 'continuous_module'

In [31]:
# Define XLNetConfig class and set default parameters for HF XLNet config  
transformer_config = tr.XLNetConfig.build(
    d_model=d_model, n_head=4, n_layer=2, total_seq_length=max_sequence_length
)
# Define the model block including: inputs, masking, projection and transformer block.
body = tr.SequentialBlock(
    inputs, tr.MLPBlock([d_model]), tr.TransformerBlock(transformer_config, masking=inputs.masking)
)

# Define the evaluation top-N metrics and the cut-offs
metrics = [NDCGAt(top_ks=[20, 40], labels_onehot=True),  
           RecallAt(top_ks=[20, 40], labels_onehot=True)]

# Define a head related to next item prediction task 
head = tr.Head(
    body,
    tr.NextItemPredictionTask(weight_tying=True, 
                              metrics=metrics),
    inputs=inputs,
)

# Get the end-to-end Model class 
model = tr.Model(head)

### Train Model

In [33]:
per_device_train_batch_size: int = 128
per_device_eval_batch_size: int = 32

In [34]:
from transformers4rec.config.trainer import T4RecTrainingArguments
from transformers4rec.torch import Trainer
# Set hyperparameters for training 
train_args = T4RecTrainingArguments(data_loader_engine='merlin', 
                                    dataloader_drop_last = True,
                                    gradient_accumulation_steps = 1,
                                    per_device_train_batch_size = per_device_train_batch_size, 
                                    per_device_eval_batch_size = per_device_eval_batch_size,
                                    output_dir = "./tmp", 
                                    learning_rate=0.0005,
                                    lr_scheduler_type='cosine', 
                                    learning_rate_num_cosine_cycles_by_epoch=1.5,
                                    num_train_epochs=5,
                                    max_sequence_length=20, 
                                    report_to = [],
                                    logging_steps=50,
                                    no_cuda=False)

In [35]:
# Instantiate the T4Rec Trainer, which manages training and evaluation for the PyTorch API
trainer = Trainer(
    model=model,
    args=train_args,
    schema=schema,
    compute_metrics=True,
)

Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [ ]:

train_paths = f"./data/ebnerd_demo/train/history.parquet"
eval_paths = f"./data/ebnerd_demo/validatn/history.parquet"
print(train_paths)

trainer.train_dataset_or_path = train_paths
trainer.reset_lr_scheduler()
trainer.train()
trainer.state.global_step += 1
print('finished')

# Evaluate on the following day
trainer.eval_dataset_or_path = eval_paths
train_metrics = trainer.evaluate(metric_key_prefix='eval')

for key in sorted(train_metrics.keys()):
    print(" %s = %s" % (key, str(train_metrics[key]))) 

wipe_memory()